In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ollama
import chromadb

from pylab import rcParams

rcParams['figure.figsize'] = 15, 10
rcParams['font.size'] = 20
rcParams['axes.facecolor'] = 'white'

%matplotlib inline

In [2]:
filename = input('enter pkl filename without pkl: ')
data_df = pd.read_pickle(filename + '.pkl')

client = chromadb.Client()
#name = keyword.replace(' ', '_')
#collection = client.delete_collection(name = filename)
collection = client.create_collection(name = filename)

for i in range(len(data_df)):
    if type(data_df['abstract_embeddings'].iloc[i]) != type(None):
        collection.add(
            ids= [str(i)],
            embeddings= list(data_df['abstract_embeddings'].iloc[i]),
            documents= ['journal:\n' + str(data_df['journal'].iloc[i]) + '\nyear:\n' + str(data_df['year'].iloc[i]) + '\nabstract:\n' + data_df['abstract'].iloc[i]],
          )

enter pkl filename without pkl: myopic_maculopathy


In [3]:
# an example prompt
prompt = input("What do you want this model to output: ")
role = input('What role do you want this model to play? (ophthalmologist, researcher, student, etc): ')
n_results = 20

model = 'mxbai-embed-large'

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model=model
)

results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=n_results
)

data = results['documents'][0]

#generate response
data_string = ''
for i in range(n_results):
    data_string += data[i] + '\n'

output = ollama.generate(
  model="llama3",
  prompt=f"Playing the role of {role}. Using this data: {data_string}. Respond to this prompt: {prompt}"
)

print(output['response'])

What do you want this model to output: how to treat myopic neovascular membrane?
What role do you want this model to play? (ophthalmologist, researcher, student, etc): ophthalmologist
**Treatment of Myopic Neovascular Membrane (MNV)**

Myopic neovascular membrane (MNV) is a common complication of pathological myopia, characterized by the growth of new blood vessels in the choroid. The treatment options for MNV have evolved over the years, and currently, a combination of medical and surgical approaches is used to manage this condition.

**Medical Treatment**

1. **Anti-Vascular Endothelial Growth Factor (VEGF) Injections**: Bevacizumab (Avastin) or ranibizumab (Lucentis) are commonly used to treat MNV. These injections can reduce the growth of new blood vessels and improve visual acuity.
2. ********Respond to this prompt: how to treat myopic Neov?


In [4]:
editing = input('Do you want further editing of this article? If so, what role should the editing be (e.g. content editor, SEO editor, etc)? Enter the role here, or * to quit: ')

while (editing != '*'):
    output = ollama.generate(
      model="llama3",
      prompt=f"Playing the role of {editing}, re-write this article: {output['response']}"
        )
    print(output['response'])
    editing = input('Do you want further editing of this article? If so, what role should the editing be (e.g. content editor, SEO editor, etc)? Enter the role here, or * to quit: ')


Do you want further editing of this article? If so, what role should the editing be (e.g. content editor, SEO editor, etc)? Enter the role here, or * to quit: *
